In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = "/content/drive/MyDrive/Yoga Pose/Dataset"

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Accuracy
import numpy as np

In [ ]:
xcep = Xception(include_top=False, weights="imagenet", input_shape=[224,224,3])

83697664/83683744 [==============================] - 0s 0us/step


In [ ]:
for layer in xcep.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(xcep.output)

In [ ]:
prediction  = Dense(5, activation='softmax')(x)

In [ ]:
model = Model(inputs = xcep.input, outputs=prediction)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [ ]:
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics = 'Accuracy')

In [ ]:
data = ImageDataGenerator(horizontal_flip=True, shear_range=0.2,zoom_range=0.2,rescale=1./255, validation_split=0.3)

In [ ]:
training_set = data.flow_from_directory(dataset,
                                           target_size=(224,224),
                                           batch_size=16,
                                           subset='training',
                                           class_mode='categorical')

Found 694 images belonging to 5 classes.


In [ ]:
testing_set = data.flow_from_directory(dataset,
                                        target_size=(224,224),
                                           batch_size=16,
                                           subset='validation',
                                           class_mode='categorical')

Found 294 images belonging to 5 classes.


In [ ]:
len(training_set), len(testing_set)

(44, 19)

In [ ]:
model.fit(training_set,
          validation_data = testing_set,
          epochs =26,
          steps_per_epoch=len(training_set)//16,
          validation_steps = len(testing_set)//16)


Epoch 1/26
2/2 [==============================] - 10s 7s/step - loss: 0.6638 - Accuracy: 0.9375 - val_loss: 0.1552 - val_Accuracy: 0.9375
Epoch 2/26
2/2 [==============================] - 10s 7s/step - loss: 0.1150 - Accuracy: 0.9688 - val_loss: 1.7121 - val_Accuracy: 0.8750
Epoch 3/26
2/2 [==============================] - 12s 6s/step - loss: 0.1096 - Accuracy: 0.9688 - val_loss: 2.1417 - val_Accuracy: 0.6875
Epoch 4/26
2/2 [==============================] - 10s 6s/step - loss: 0.0560 - Accuracy: 0.9688 - val_loss: 1.9608 - val_Accuracy: 0.8750
Epoch 5/26
2/2 [==============================] - 10s 6s/step - loss: 0.9275 - Accuracy: 0.9062 - val_loss: 2.7884 - val_Accuracy: 0.8125
Epoch 6/26
2/2 [==============================] - 10s 6s/step - loss: 0.3086 - Accuracy: 0.9688 - val_loss: 3.1315 - val_Accuracy: 0.8125
Epoch 7/26
2/2 [==============================] - 10s 6s/step - loss: 0.4234 - Accuracy: 0.9375 - val_loss: 0.6619 - val_Accuracy: 0.8750
Epoch 8/26
2/2 [==================

In [ ]:
model.save('xcep_yoga.h5')

In [ ]:
model = load_model('xcep_yoga.h5')

In [ ]:
img = load_img("/content/drive/MyDrive/Yoga Pose/Dataset/Plank/00000000.jpg", target_size=(224,224))

In [ ]:
img = image.img_to_array(img)

In [ ]:
img.shape

(224, 224, 3)

In [ ]:
x = np.expand_dims(img, axis=0)
img_data=preprocess_input(x)
img_data.shape


(1, 224, 224, 3)

In [ ]:
pred = model.predict(img_data)

In [ ]:
p = np.argmax(pred)

In [ ]:
columns = ['Downdog', 'Goddess', 'Plank', 'Tree', 'Warrior2']

In [ ]:
result = str(columns[p])

In [ ]:
result

'Plank'